In [2]:
from inflation_analysis import grouping

In [3]:
start_year = 2021
end_year = 2022
years = range(start_year, end_year+1)
base_year = start_year
group_mmb = None 
factor = 1
cex_data_folder = '/Users/roykisluk/Downloads/Consumer_Expenditure_Survey/' 
folder_names_pathname = 'Data_clean/CEX_folder_names.csv' 
prodcode_dict_pathname = 'Data_clean/prodcode_dictionary_c3-c399.csv'

import pandas as pd
import pyreadstat
import numpy as np 


In [4]:
nationality, observance, income, ses, age, family_size, total_misparmb = grouping(start_year, end_year)

In [5]:
groups = nationality

In [6]:
print(groups)

{'Arab': {2021:       s_seker  misparmb  hodesh_hakira  shnat_hakira  quarter    weight  \
28     2021.0   28135.0            2.0        2021.0      1.0   532.106   
46     2021.0   28153.0            2.0        2021.0      1.0   211.650   
47     2021.0   28154.0            2.0        2021.0      1.0  1094.311   
48     2021.0   28155.0            2.0        2021.0      1.0    85.031   
49     2021.0   28156.0            2.0        2021.0      1.0   818.063   
...       ...       ...            ...           ...      ...       ...   
6043   2021.0   53447.0           12.0        2021.0      4.0   111.567   
6044   2021.0   53448.0           12.0        2021.0      4.0    60.408   
6047   2021.0   53577.0            1.0        2022.0      4.0    40.447   
6049   2021.0   53579.0            1.0        2022.0      4.0   312.570   
6054   2021.0   58002.0            8.0        2021.0      3.0   604.582   

      nefashot  nefeshstandartit  mefarnesim  nefashot0_4  ...     c34  \
28       

In [7]:
if base_year is None:   
    base_year = start_year
years = range(start_year, end_year + 1)

groups_mmb = {key: {} for key in groups.keys()}
for key in groups:
    for year in years:
        groups_mmb[key][year] = groups[key][year][['misparmb']]

group_analysis = {}
for key in groups.keys():
    group_number = list(groups.keys()).index(key) + 1
    total_groups = len(groups)
    print(groups_mmb[key])

{2021:       misparmb
28     28135.0
46     28153.0
47     28154.0
48     28155.0
49     28156.0
...        ...
6043   53447.0
6044   53448.0
6047   53577.0
6049   53579.0
6054   58002.0

[951 rows x 1 columns], 2022:       misparmb
4      57093.0
7      57096.0
8      57097.0
51     57322.0
126    57397.0
...        ...
5460   69757.0
5461   69758.0
5463   69767.0
5464   69768.0
5465   69769.0

[727 rows x 1 columns]}
{2021:       misparmb
0      28106.0
1      28107.0
2      28108.0
3      28109.0
4      28110.0
...        ...
6051   53759.0
6052   53760.0
6053   53762.0
6055   58031.0
6056   65289.0

[4911 rows x 1 columns], 2022:       misparmb
0      57089.0
1      57090.0
2      57091.0
3      57092.0
5      57094.0
...        ...
5473   69827.0
5474   80031.0
5475   80032.0
5476   80033.0
5477   83443.0

[4584 rows x 1 columns]}


In [8]:
group_mmb = groups_mmb['Arab']

# start

In [ ]:

years = range(start_year, end_year + 1)

# Load folder names
folder_names_df = pd.read_csv(folder_names_pathname)

# Functions

# Aggregate total consumption for this group
def total_consumption_value(df): 
    total_consumption = 0.0
    for j in range(0, len(df)):
        total_consumption += df['mehir'][j]
    return total_consumption

# Keep only shared prodcodes for both dataframes (years)
def keep_shared_prodcodes(df1, df2):
    shared_prodcodes = set(df1['prodcode']).intersection(set(df2['prodcode']))
    df1_shared = df1[df1['prodcode'].isin(shared_prodcodes)].reset_index(drop=True)
    df2_shared = df2[df2['prodcode'].isin(shared_prodcodes)].reset_index(drop=True)
    return df1_shared, df2_shared

# Calculate weights for each product
def weighting(df):
    weights = pd.DataFrame(df['prodcode'].unique(), columns=['prodcode'])
    weights['weight'] = 0.0
    total_consumption = total_consumption_value(df)
    for j in range(0, len(weights)):
        weights.loc[j, 'weight'] = df[df['prodcode'] == weights.loc[j, 'prodcode']]['mehir'].sum() / total_consumption
    return weights

# Calculate average price for each product
def average_price(df):
    average_prices = pd.DataFrame(df['prodcode'].unique(), columns=['prodcode'])
    average_prices['price'] = 0.0
    for j in range(0, len(average_prices)):
        average_prices.loc[j, 'price'] = (df[df['prodcode'] == average_prices.loc[j, 'prodcode']]['mehir'] / df[df['prodcode'] == average_prices.loc[j, 'prodcode']]['kamut']).mean()
    return average_prices

# Calculate Laspeyres index
def Laspeyres(df_base, df_current):
    # Create index dataframe for each product
    index_df = pd.DataFrame(df_base['prodcode'].unique(), columns=['prodcode'])
    # Set index to 0
    index_df['index'] = 0.0
    # Calculate weights for each product
    weights = weighting(df_base)
    # Calculate average prices for each product, base year and current year
    average_prices_base = average_price(df_base)
    average_prices_current = average_price(df_current)
    # Merge dataframes
    index_df = index_df.merge(weights, on='prodcode', how='left')
    index_df = index_df.merge(average_prices_base, on='prodcode', how='left', suffixes=('', '_base'))
    index_df = index_df.merge(average_prices_current, on='prodcode', how='left', suffixes=('_base', '_current'))
    # Set total index for current year, for this group, to 0
    total_index = 0.0
    # Calculate index for each product
    for j in range(len(index_df)):
        price_current = index_df.loc[j, 'price_current']
        price_base = index_df.loc[j, 'price_base']
        # If price is 0, NaN or Inf, set index to factor * 100 (default is 1: index 100 for missing products, i.e. assume no change)
        if price_base == 0 or pd.isna(price_base) or np.isinf(price_base):
            index_df.loc[j, 'index'] = factor * 100
            continue
        if price_current == 0 or pd.isna(price_current) or np.isinf(price_current):
            index_df.loc[j, 'index'] = factor * 100
            continue
        index_df.loc[j, 'index'] = (price_current / price_base) * 100
    # Calculate yearly price index for this group
    for j in range(len(index_df)):
        weight = index_df.loc[j, 'weight']
        total_index += weight * index_df.loc[j, 'index']
    return index_df, total_index

# Merge secondary categories (3 digits) to primary categories (2 digits)
def merge_to_primary(df):
    df['prodcode_primary'] = df['prodcode'].astype(str).str[:2]
    grouped = df.groupby('prodcode_primary', group_keys=False).apply(
        lambda x: pd.Series({
            'price_index': np.average(x['price_index'], weights=x['weight']) if x['weight'].sum() > 0 else np.nan,
            'total_weight': x['weight'].sum()
        }),
        include_groups=False
    ).reset_index()
    grouped.rename(columns={'prodcode_primary': 'prodcode'}, inplace=True)
    grouped.rename(columns={'total_weight': 'weight'}, inplace=True)
    return grouped

In [10]:
# Load survey data for each year
dfs_survey = {}
for year in years:
    subfolder = folder_names_df.loc[folder_names_df['Year'] == year, 'Folder_Name'].values[0]
    data_prices_pathname = f"{cex_data_folder}{subfolder}/{subfolder}datayoman.sas7bdat"
    df, meta = pyreadstat.read_sas7bdat(data_prices_pathname)
    df.columns = df.columns.str.lower()
    dfs_survey[year] = df

In [11]:
print(dfs_survey[2021]['misparmb'].nunique())

6024


In [12]:
print(group_mmb)

{2021:       misparmb
28     28135.0
46     28153.0
47     28154.0
48     28155.0
49     28156.0
...        ...
6043   53447.0
6044   53448.0
6047   53577.0
6049   53579.0
6054   58002.0

[951 rows x 1 columns], 2022:       misparmb
4      57093.0
7      57096.0
8      57097.0
51     57322.0
126    57397.0
...        ...
5460   69757.0
5461   69758.0
5463   69767.0
5464   69768.0
5465   69769.0

[727 rows x 1 columns]}


In [13]:
# Filter observations for relevant group
if group_mmb is not None:
    for year in years:
        dfs_survey[year] = dfs_survey[year][dfs_survey[year]['misparmb'].isin(group_mmb[year]['misparmb'])]



In [14]:
print(dfs_survey[2021]['misparmb'].nunique())

947


In [15]:
print(dfs_survey[2021]['prodcode'].min())
print(dfs_survey[2021]['prodcode'].max())

212019.0
442020.0


In [16]:

# Filter observations with prodcode that starts with 3
for year in years:
    dfs_survey[year] = dfs_survey[year][dfs_survey[year]['prodcode'].astype(str).str.startswith('3')].reset_index(drop=True)
    dfs_survey[year]['prodcode'] = dfs_survey[year]['prodcode'].astype(str).str[:3]

In [17]:
print(dfs_survey[2021]['prodcode'].min())
print(dfs_survey[2021]['prodcode'].max())

300
398


In [19]:
print(dfs_survey[2021]['prodcode'].nunique())

64


In [ ]:

# Calculate weights and price indexes
yearly_price_index = {}
df_price_index = {}
for year in years:
    df_base, df_current = keep_shared_prodcodes(dfs_survey[base_year], dfs_survey[year])
    df_price_index[year], yearly_price_index[year] = Laspeyres(df_base, df_current)

# Combine all years into a single dataframe
combined_df = pd.concat(df_price_index.values(), keys=df_price_index.keys(), names=['Year', 'Index']).reset_index(level='Index', drop=True).reset_index()
combined_df = combined_df[['Year', 'prodcode', 'index', 'weight']]

# Merge to secondary and primary categories
df_primary = {}
for year in years:
    df_primary[year] = merge_to_primary(combined_df[year])

# Combine secondary and primary categories into a single dataframe
combined_primary_df = pd.concat(df_primary.values(), keys=df_primary.keys(), names=['Year', 'Index']).reset_index(level='Index', drop=True).reset_index()

# Keep only the necessary columns
combined_primary_df = combined_primary_df[['Year', 'prodcode', 'price_index', 'weight']]

# Load prodcode dictionary
prodcode_dict_df = pd.read_csv(prodcode_dict_pathname)

# Remove description column if it already exists
if 'description' in combined_primary_df.columns:
    combined_primary_df = combined_primary_df.drop(columns=['description'])

# Convert prodcode to string in both dataframes before merging
prodcode_dict_df['prodcode'] = prodcode_dict_df['prodcode'].astype(str)